## Лабораторное занятие №6. Логистическая регрессия

Предлагается спрогнозировать, купит клиент велосипед или нет

In [1]:
# подключим библиотеки
# Импорт библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# считаем данные
data = pd.read_csv('data/bike_buyers_clean.csv')

In [3]:
# выводим первые 5 строк датафрейма 
data.head()

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [5]:
# смотрим размер датафрейма
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                1000 non-null   int64 
 1   Marital Status    1000 non-null   object
 2   Gender            1000 non-null   object
 3   Income            1000 non-null   int64 
 4   Children          1000 non-null   int64 
 5   Education         1000 non-null   object
 6   Occupation        1000 non-null   object
 7   Home Owner        1000 non-null   object
 8   Cars              1000 non-null   int64 
 9   Commute Distance  1000 non-null   object
 10  Region            1000 non-null   object
 11  Age               1000 non-null   int64 
 12  Purchased Bike    1000 non-null   object
dtypes: int64(5), object(8)
memory usage: 101.7+ KB


Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом describe из библиотеки pandas со значением параметра include = 'object'.

In [ ]:
data.describe(include='object')

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


Закодируйте все категориальные столбцы с двумя категориями следующим образом: самая часто встречающаяся категория превращается в 1, другая в 0.

In [12]:
from sklearn.preprocessing import LabelEncoder

In [14]:
data_copy = data.copy()

binary_columns = [col for col in data_copy.select_dtypes(include='object').columns if data_copy[col].nunique() == 2]

for col in binary_columns:
    # Находим самую частую категорию и кодируем её как 1, а вторую категорию - как 0
    le = LabelEncoder()
    data_copy[col] = le.fit_transform(data_copy[col])

Удалите остальные категориальные столбцы

In [15]:
data_copy = data_copy.drop(columns=[col for col in data_copy.select_dtypes(include='object').columns if data_copy[col].nunique() > 2])

In [17]:
data_copy.head()

,ID,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Purchased Bike
0,12496,0,0,40000,1,1,0,42,0
1,24107,0,1,30000,3,1,1,43,0
2,14177,0,1,80000,5,0,2,60,0
3,24381,1,1,70000,0,1,1,41,1
4,25597,1,1,30000,0,0,0,36,1


Удалите столбец ID, так как он по сути нечисловой

In [20]:
data_copy.drop(columns='ID')

,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Purchased Bike
0,0,0,40000,1,1,0,42,0
1,0,1,30000,3,1,1,43,0
2,0,1,80000,5,0,2,60,0
3,1,1,70000,0,1,1,41,1
4,1,1,30000,0,0,0,36,1
...,...,...,...,...,...,...,...,...
995,0,1,60000,2,1,2,54,1
996,1,1,70000,4,1,0,35,1
997,0,1,60000,2,1,0,38,1
998,1,1,100000,3,0,3,38,0


Разбейте данные на матрицу признаков X и вектор y с целевой переменной (Purchased Bike)

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [22]:
X = data_copy.drop(columns='Purchased Bike')
y = data_copy['Purchased Bike']

Разбейте данные на тренировочную и тестовую части, в тест отправьте 30% данных. Зафиксируйте random_state = 42

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Отмасштабируйте матрицу X, так как линейные модели чувствительны к масштабу данных

In [24]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Обучите модель логистической регрессии LogisticRegression на тренировочных данных

In [25]:
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

LogisticRegression(random_state=42)

Сделайте предсказания на тренировочных и на тестовых данных

In [26]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

Оцените значение accuracy на трейне и на тесте.

In [32]:
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'train: {train_accuracy}\n test: {test_accuracy}')

train: 0.6457142857142857
 test: 0.5766666666666667


Попробуйте добавить новые признаки в модель, используя PolynomialFeatures. Создайте полиномиальные признаки degree = 2

In [33]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (accuracy)

In [36]:
model_poly = LogisticRegression(max_iter=1000, random_state=42)
model_poly.fit(X_train_poly, y_train)

y_train_poly_pred = model_poly.predict(X_train_poly)
y_test_poly_pred = model_poly.predict(X_test_poly)

train_accuracy_poly = accuracy_score(y_train, y_train_poly_pred)
test_accuracy_poly = accuracy_score(y_test, y_test_poly_pred)

print(f'train_poly: {train_accuracy_poly}\n test_poly: {test_accuracy_poly}')

train_poly: 0.69
 test_poly: 0.6166666666666667


Появились новые требования от заказчика!

Заказчик просит, чтобы:

доля найденных моделью потенциальных покупателей была максимальной
accuracy при этом была не ниже, чем 0.6 (отклонения accuracy на тестовых данных на  ±0.05  допустимы)

Сначала посмотрите, какие значения recall и accuracy имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом predict)

In [39]:
from sklearn.metrics import recall_score

In [55]:
# Предсказания вероятностей
y_probs = model.predict_proba(X_test)[:, 1]

# Изменение порога
threshold = 0.45  # Попробуйте различные значения
y_pred_adjusted = (y_probs >= threshold).astype(int)

# Оценка
accuracy = accuracy_score(y_test, y_pred_adjusted)
print("Adjusted Accuracy:", accuracy)

# Вычисление доли найденных покупателей
recall = recall_score(y_test, y_pred_adjusted)
print("Recall (Proportion of Identified Buyers):", recall)

Adjusted Accuracy: 0.61
Recall (Proportion of Identified Buyers): 0.5921052631578947


Идентефицировано **59%** потенциальных покупателей, это при минимальной accuracy в **61%**

Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики

При помощи подбора порога необходимо сильно увеличить значение recall.
Значение  accuracy может уменьшиться в допустимых рамках.

Приоритетом для заказчика является значительное увеличение полноты.

In [60]:
y_probs = model.predict_proba(X_test)[:, 1]

thresholds = np.arange(0.1, 1.0, 0.01)
results = []

for threshold in thresholds:
    y_pred_adjusted = (y_probs >= threshold).astype(int)
    
    accuracy = accuracy_score(y_test, y_pred_adjusted)
    recall = recall_score(y_test, y_pred_adjusted)
    
    results.append((threshold, accuracy, recall))

results_df = pd.DataFrame(results, columns=['Threshold', 'Accuracy', 'Recall'])

filtered_results = results_df[results_df['Accuracy'] >= 0.55]

# ! Поиск порога с максимальной полнотой среди отфильтрованных результатов
if not filtered_results.empty:
    best_recall_row = filtered_results.loc[filtered_results['Recall'].idxmax()]
    best_threshold = best_recall_row['Threshold']
    best_recall = best_recall_row['Recall']
    best_accuracy = best_recall_row['Accuracy']

    print("Optimal Threshold for Maximum Recall with Accuracy >= 0.55:")
    print("Best threshold:", best_threshold)
    print("Recall at this threshold:", best_recall)
    print("Accuracy at this threshold:", best_accuracy)
else:
    print("No valid threshold found with accuracy >= 0.55.")

Optimal Threshold for Maximum Recall with Accuracy >= 0.55:
Best threshold: 0.23999999999999994
Recall at this threshold: 0.9539473684210527
Accuracy at this threshold: 0.55


Порог `0.24` значительно увеличивает значение `recall` до **95.39%**, что является отличным результатом. Это значит, что модель практически идентифицирует всех потенциальных покупателей, но при этом `accuracy` немного ниже **0.6**, что соответствует требованиям заказчика по допустимым отклонениям.